**Import des libraires**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

Dans ce TP, nous allons travailleurs sur les données *Titanic*. Ces données contiennent des variables décrivant les passagers du Titanic et la variable d'intérêt que l'on va chercher à prédire est celle qui indique leur survie (apellée *Survived* dans le fichier).

Pour plus de précisions sur ces données, on pourra consulter : 
https://www.kaggle.com/c/titanic/data


1. Dans un premier temps, on va regarder le type de chacunes des variables et si certaines observations sont manquantes.

In [ ]:
titanic = pd.read_csv('https://gist.githubusercontent.com/michhar/2dfd2de0d4f8727f873422c5d959fff5/raw/fa71405126017e6a37bea592440b4bee94bf7b9e/titanic.csv')

X = titanic.drop('Survived', axis = 1)
y = titanic.Survived


In [ ]:
# types de variables
# données manquantes 


Comme indiqué précédemment, on cherche à prédire la classe (*Survived*) en fonction des autres variables.

In [ ]:
X = titanic.drop('Survived', axis = 1)
y = titanic.Survived

On va tout d'abord s'intéresser aux données numériques.

2. Pour gérer le problème des valeurs manquantes, effectuer une imputation (par la moyenne ou la médiane) en utilisant *SimpleImputer* puis appliquer une régression logistique.
3. Après cette imputation, on va standardiser les données via *StandardScaler*.

In [ ]:
# seperation des variables suivant leurs types 
numeric_features = ['Age', 'SibSp', 'Parch', 'Fare']
categorical_features = ['Pclass', 'Sex', 'Embarked']

X_num= X[numeric_features].copy()

X_train, X_holdout, y_train, y_holdout = train_test_split(X_num, y, stratify = y, test_size = 0.2, random_state = 42)


#imputation des données manquantes 

# standardisation des données 

clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred= clf.predict(X_holdout)

sum(y_pred==y_holdout)/len(y_holdout)


On va tout maintenant s'intéresser aux données catégorielles.

4. Pour gérer le problème des valeurs manquantes, effectuer une imputation (en utilisant les bons paramètres pour ce type de données).
5. Après cette imputation, on va appliquera un *One Hot Encoding* les données avant d'appliquer une simple régression logistique.

In [ ]:
X_cat= X[categorical_features].copy()

X_train, X_holdout, y_train, y_holdout = train_test_split(X_cat, y, stratify = y, test_size = 0.2, random_state = 42)

#imputation 


#one hot encoder

clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred= clf.predict(X_holdout)

sum(y_pred==y_holdout)/len(y_holdout)


Les pre-processing utilisés ainsi que le classifieur ont tous des paramètres qu'il conviendrait de régler correctement. 

6. Reprenons notre chaîne de traitement des données numériques et cette fois, nous allons créer un objet *Pipeline* qui contiendra nos étapes d'imputation, de standardisation et de classification.
On cherchera à trouver la meilleure valeur C (pour la régression logisitque sur la grille [0.01, 0.1, 1, 10, 100]), si la standardisation est nécessaire ou pas et enfin si il est préférable d'utiliser la moyenne ou la médiane pour l'imputation.

In [ ]:
# donnees numériques

X_train, X_holdout, y_train, y_holdout = train_test_split(X_num, y, stratify = y, test_size = 0.2, random_state = 42)


# Pipeline : imputer -> scaler -> classifieur
# dans un pipeline, un élément doit avoir des fonctions fit et transform
# la fonction fit est appelée et les données transformées sont données en entrée de l'élement suivant
# ex : pipeline = Pipeline ([ ('preproc', normalize()), ('clf', LogisticRegression( ))])

pipeline 


# les paramètres p d'un élement e du pipeline est identifié par la chaine de caractères 'e__p" et ensuite un nparray précise les valeurs possibles
# ex : params = [{'clf__C': [0.01, 0.1, 1, 10, 100]}


params 



rskf = StratifiedKFold(n_splits=10, random_state=42, shuffle=False)
cv = GridSearchCV(pipeline, params, cv = rskf, scoring = 'accuracy', n_jobs = -1)

cv.fit(X_train, y_train)

print(f'Best accuracy -score: {cv.best_score_:.3f}\n')
print(f'Best parameter set: {cv.best_params_}\n')
print(f'Scores: {classification_report(y_train, cv.predict(X_train))}')

preds = cv.predict(X_holdout)
print(f'Scores: {classification_report(y_holdout, preds)}\n')
print(f'accuracy score: {accuracy_score(y_holdout, preds):.3f}')

Maintent que l'on sait créer et valider un pipeline, on va voir que ceux-ci peuvent se combiner.

5. A l'aide de *ColumnTransformer*, on va appliquer les pipelins précédents sur chaque sous-ensemble de variables (numériques et catégorielles) et les combiner. Cette objet sera alors mis lui-même dans un pipeline.

In [ ]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, stratify = y, test_size = 0.2, random_state = 42)

# pipeline (preprocessing uniquement) sur les donnes catégorielles
categorical_features = ['Pclass', 'Sex', 'Embarked']
categorical_transformer = Pipeline

# pipeline (preprocessing uniquement) sur les données numériques 
numeric_features = ['Age', 'SibSp', 'Parch', 'Fare']
numeric_transformer = Pipeline(
    [
        ('imputer_num', SimpleImputer()),
        ('scaler', StandardScaler())
    ]
)

# chaque pipeline s'applique sur un sous-ensemble des variables
preprocessor = ColumnTransformer(
    [
        ('categoricals', categorical_transformer, categorical_features),
        ('numericals', numeric_transformer, numeric_features)
    ],
    remainder = 'drop'
)

# on crée un pipeline combinat l'objet preprocessor et un classifieur
pipeline 


params 





rskf = StratifiedKFold(n_splits = 10)

cv = GridSearchCV(pipeline, params, cv = rskf, scoring = 'accuracy', n_jobs = -1)

cv.fit(X_train, y_train)

print(f'Best accuracy-score: {cv.best_score_:.3f}\n')
print(f'Best parameter set: {cv.best_params_}\n')
print(f'Scores: {classification_report(y_train, cv.predict(X_train))}')

preds = cv.predict(X_holdout)
print(f'Scores: {classification_report(y_holdout, preds)}\n')
print(f'accuracy-score: {accuracy_score(y_holdout, preds):.3f}')

Pour aller plus loin, on pourra introduire une étape de RFE dans le pipeline précédent.

On pourra aussi appliquer cette méthodologie sur d'autres données (winequality-red.csv ou bien IMDB 5000 Movie)
